# MRI Lesion Segmentation Deep Dive

This notebook is the flagship entrypoint for the MRI segmentation project.
It drives experiment execution, metric analysis, and qualitative result review.


## Current Run Snapshot

- Subset run (16 cases, 4 epochs) best val Dice: **0.3750**
- Files: `results/metrics.csv`, `results/subset_run_summary.txt`
- Curves: `assets/dice_curve_subset.png`, `assets/loss_curve_subset.png`


## 1) Environment and Paths

Assumes project root:
- `/Users/maguotong/Desktop/github项目_clean/mri-lesion-segmentation-isles2022`


In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path('/Users/maguotong/Desktop/github项目_clean/mri-lesion-segmentation-isles2022')
SRC_DIR = PROJECT_ROOT / 'src'
RESULTS_DIR = PROJECT_ROOT / 'results'
MODELS_DIR = PROJECT_ROOT / 'models'
DATA_DIR = PROJECT_ROOT / 'data' / 'train'

print('Project root:', PROJECT_ROOT)
print('Data dir exists:', DATA_DIR.exists())
print('Results dir:', RESULTS_DIR)


## 2) Train the Improved 3D Residual U-Net

Run in terminal from project root:

```bash
PYTHONPATH=src python3 -m mri_seg.train \
  --data-dir data/train \
  --epochs 60 \
  --batch-size 2 \
  --patch-size 96 96 96 \
  --patches-per-volume 8 \
  --learning-rate 1e-3 \
  --positive-patch-ratio 0.7
```


In [ ]:
# Optional: trigger training directly from notebook
import os
cmd = (
    "cd /Users/maguotong/Desktop/github项目_clean/mri-lesion-segmentation-isles2022 && "
    "PYTHONPATH=src python3 -m mri_seg.train "
    "--data-dir data/train --epochs 60 --batch-size 2 "
    "--patch-size 96 96 96 --patches-per-volume 8 "
    "--learning-rate 1e-3 --positive-patch-ratio 0.7"
)
print(cmd)
# os.system(cmd)


## 3) Load and Plot Training Metrics


In [ ]:
metrics_path = RESULTS_DIR / 'metrics.csv'
if metrics_path.exists():
    df = pd.read_csv(metrics_path)
    display(df.tail())
else:
    print('metrics.csv not found. Run training first.')


In [ ]:
if metrics_path.exists():
    fig, ax = plt.subplots(1, 3, figsize=(15, 4))

    ax[0].plot(df['epoch'], df['train_dice'], label='train_dice')
    ax[0].plot(df['epoch'], df['val_dice'], label='val_dice')
    ax[0].set_title('Dice')
    ax[0].legend()

    ax[1].plot(df['epoch'], df['train_loss'], label='train_loss')
    ax[1].plot(df['epoch'], df['val_loss'], label='val_loss')
    ax[1].set_title('Loss')
    ax[1].legend()

    ax[2].plot(df['epoch'], df['val_iou'], label='val_iou')
    ax[2].plot(df['epoch'], df['val_recall'], label='val_recall')
    ax[2].set_title('Val IoU / Recall')
    ax[2].legend()

    plt.tight_layout()
    plt.show()


## 4) Inference Command

```bash
PYTHONPATH=src python3 -m mri_seg.infer \
  --checkpoint models/best_model.pt \
  --input-dir data/infer \
  --output-dir results/predictions \
  --threshold 0.5
```


## 5) Experiment Tracking Plan

- Baseline: Dice+BCE, patch size 96, positive ratio 0.7
- Ablation A: Tversky loss (`--use-tversky`)
- Ablation B: larger patch size (112 or 128, if memory allows)
- Ablation C: threshold sweep (0.3 to 0.7)

Keep all runs in `results/metrics.csv` snapshots and compare best validation Dice.
